In [ ]:
from transformers import pipeline
from langdetect import detect
from sentence_transformers import CrossEncoder

class GrievanceSummarizer:
    def __init__(self):
        self.models = {
            'en': pipeline("summarization", model="facebook/bart-large-cnn"),
            'hi': pipeline("summarization", model="ai4bharat/MultiIndicSentenceSummarization"),
            'ta': pipeline("summarization", model="ai4bharat/MultiIndicSentenceSummarization"),
        }

        self.simplifier = CrossEncoder('cross-encoder/quora-distilroberta-base')

    def detect_language(self, text):
        try:
            lang = detect(text)
            return 'hi' if lang == 'hi' else lang
        except:
            return 'en'

    def summarize(self, text, max_length=50):
        lang = self.detect_language(text)
        summary = self.models[lang](text, max_length=max_length, min_length=30, do_sample=False)[0]['summary_text']
        return self.simplify(summary)

    def simplify(self, text, threshold=0.7):
        # Semantic similarity check
        simplified_versions = [
            self._rule_based_simplify(text),
            self._model_based_simplify(text)
        ]

        # Select best simplification
        scores = [self.simplifier.predict((text, s)) for s in simplified_versions]
        best_idx = scores.index(max(scores))
        return simplified_versions[best_idx] if max(scores) > threshold else text

    def _rule_based_simplify(self, text):
        # Add domain-specific simplification rules
        replacements = {
            "residential premises": "houses",
            "municipal corporation": "local office",
            "expeditious resolution": "quick fix"
        }
        for k, v in replacements.items():
            text = text.replace(k, v)
        return text

    def _model_based_simplify(self, text):
        return pipeline("text2text-generation", model="t5-small")(
            f"simplify: {text}",
            max_length=100
        )[0]['generated_text']

In [ ]:
summarizer = GrievanceSummarizer()

In [ ]:
summarizer.summarize("""सड़क पर बड़े-बड़े potholes हो गए हैं। Last week एक scooter driver गड्ढे में गिरकर seriously injured हो गया। कृपया immediate action लें।""")